In [26]:
import pandas as pd

data = pd.read_xml('Project3_data/100-01.xml')

data['TEXT'][0]
data['MEDICATION']

0   NaN
1   NaN
Name: MEDICATION, dtype: float64

The goal is to provide a frequency distribution of the vital signs/physical exam readings observed across the patient clinical notes.

What are vital signs/physical exam readings?
*   Blood pressure
*   Pulse
*   Temperature
*   Respitory rate
*   Height
*   Weight
*   SaO2 (measures oxygen saturation in arterial blood in percent)

Since we are analyzing free-text data that is only semi-structured, I have found the following aliases for each vital sign.
*   Blood pressure: BP
*   Pulse: P, HR, heart rate,
*   Temperature: T, Temp
*   Respitory rate: Respirations, RR, Fs
*   Height: Ht
*   Weight: Wt, none (only a number followed by 'pounds' or 'lbs'), weighing, W.,
*   Oxygen saturation: SaO2, O2 saturation, saturating, Sat, O2 sat